In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import normalize

In [2]:
data=pd.read_excel('dataset/ZPEHbb.xlsx',date_parser=True,)

In [3]:
data.set_index("Date",inplace=True)

In [4]:
data.index

Index(['1-Jan-2018', '2-Jan-2018', '3-Jan-2018', '4-Jan-2018', '5-Jan-2018',
       '6-Jan-2018', '7-Jan-2018', '8-Jan-2018', '9-Jan-2018', '10-Jan-2018',
       ...
       '22-Dec-2019', '23-Dec-2019', '24-Dec-2019', '25-Dec-2019',
       '26-Dec-2019', '27-Dec-2019', '28-Dec-2019', '29-Dec-2019',
       '30-Dec-2019', '31-Dec-2019'],
      dtype='object', name='Date', length=730)

In [5]:
data.index=pd.to_datetime(data.index)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 730 entries, 2018-01-01 to 2019-12-31
Data columns (total 30 columns):
apple          730 non-null int64
burger         730 non-null int64
chocolate      730 non-null int64
donuts         730 non-null int64
egg            730 non-null int64
fish           730 non-null int64
garlic         730 non-null int64
honey          730 non-null int64
ice cream      730 non-null int64
jelly          730 non-null int64
ketchup        730 non-null int64
lemons         730 non-null int64
mango          730 non-null int64
oranges        730 non-null int64
pasta          730 non-null int64
yougurt        730 non-null int64
quicee         730 non-null int64
carrot         730 non-null int64
rice           729 non-null float64
olive oil      730 non-null int64
yam            730 non-null int64
popcorn        730 non-null int64
nut            730 non-null int64
tomato         730 non-null int64
vegetables     730 non-null int64
peri sauces    730 non-n

In [7]:
data.shape

(730, 30)

In [8]:
data.isnull().sum()


apple          0
burger         0
chocolate      0
donuts         0
egg            0
fish           0
garlic         0
honey          0
ice cream      0
jelly          0
ketchup        0
lemons         0
mango          0
oranges        0
pasta          0
yougurt        0
quicee         0
carrot         0
rice           1
olive oil      0
yam            0
popcorn        0
nut            0
tomato         0
vegetables     0
peri sauces    0
mutton         0
spaghetti      0
wings          0
 prac          0
dtype: int64

In [9]:
data['rice'].fillna(1, inplace=True)


In [10]:
data.isnull().sum()


apple          0
burger         0
chocolate      0
donuts         0
egg            0
fish           0
garlic         0
honey          0
ice cream      0
jelly          0
ketchup        0
lemons         0
mango          0
oranges        0
pasta          0
yougurt        0
quicee         0
carrot         0
rice           0
olive oil      0
yam            0
popcorn        0
nut            0
tomato         0
vegetables     0
peri sauces    0
mutton         0
spaghetti      0
wings          0
 prac          0
dtype: int64

In [11]:
data

,apple,burger,chocolate,donuts,egg,fish,garlic,honey,ice cream,jelly,...,yam,popcorn,nut,tomato,vegetables,peri sauces,mutton,spaghetti,wings,prac
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0,1,1,0,1,1,0,1,1,0,...,0,0,1,0,1,0,1,1,1,1
2018-01-02,0,1,1,1,1,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
2018-01-03,1,1,0,0,1,0,1,0,1,1,...,0,1,0,0,1,0,1,1,1,1
2018-01-04,1,0,0,1,1,1,1,0,1,0,...,1,1,0,0,1,0,1,0,0,1
2018-01-05,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,1,1,1,1,0,0,1,0,1,0,...,1,0,1,1,1,0,1,1,1,1
2019-12-28,0,0,1,0,1,0,0,1,1,0,...,0,1,1,1,0,1,1,0,0,1
2019-12-29,1,1,1,1,0,0,1,0,0,1,...,1,0,0,1,0,0,1,1,1,1


In [10]:
train=data.iloc[:,:550]

In [11]:
test=data.iloc[:,550:]

In [12]:
scalar=MinMaxScaler()
data=scalar.fit_transform(data)

/home/fahad/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [13]:
train_size=int(len(data)* 0.75)
test_size=len(data)-train_size
train_data,test_data=data[0:train_size],data[train_size:len(data)]
print(train_data.shape,test_data.shape)

(547, 30) (183, 30)


In [19]:
train_generator=TimeseriesGenerator(train_data,train_data,length=3, batch_size=1)

In [23]:
x,y=val_generator[0]

In [24]:
y

array([[0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1.,
        0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0.]])

In [22]:
val_generator=TimeseriesGenerator(test_data,test_data,length=3, batch_size=1)

In [64]:
model=Sequential()

In [68]:
regressior = Sequential()

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (3, 30)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 120, activation = 'relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units = 30))
optimizer = Adam(lr=1e-4)
regressior.compile(loss='mse', optimizer=optimizer,metrics=['mae'])

In [69]:
regressior.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 3, 60)             21840     
_________________________________________________________________
dropout_32 (Dropout)         (None, 3, 60)             0         
_________________________________________________________________
lstm_33 (LSTM)               (None, 3, 60)             29040     
_________________________________________________________________
dropout_33 (Dropout)         (None, 3, 60)             0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 3, 80)             45120     
_________________________________________________________________
dropout_34 (Dropout)         (None, 3, 80)             0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 120)             

In [70]:

regressior.fit_generator(train_generator,epochs=5, verbose=1,validation_data=(val_generator))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 544 steps, validate for 180 steps
Epoch 1/5
544/544 [==============================] - 8s 15ms/step - loss: 0.3534 - mae: 0.5013 - val_loss: 0.2510 - val_mae: 0.4847
Epoch 2/5
544/544 [==============================] - 5s 9ms/step - loss: 0.2595 - mae: 0.4847 - val_loss: 0.2429 - val_mae: 0.4792
Epoch 3/5
544/544 [==============================] - 5s 9ms/step - loss: 0.2534 - mae: 0.4828 - val_loss: 0.2406 - val_mae: 0.4777
Epoch 4/5
544/544 [==============================] - 5s 9ms/step - loss: 0.2481 - mae: 0.4798 - val_loss: 0.2393 - val_mae: 0.4771
Epoch 5/5
544/544 [==============================] - 5s 9ms/step - loss: 0.2484 - mae: 0.4808 - val_loss: 0.2396 - val_mae: 0.4795


In [71]:
model.predict_generator(val_generator)

Instructions for updating:
Please use Model.predict, which supports generators.


array([[[0., 1., 1., ..., 0., 1., 0.],
        [1., 1., 1., ..., 1., 0., 0.],
        [1., 1., 0., ..., 1., 0., 0.]],

       [[1., 1., 1., ..., 1., 0., 0.],
        [1., 1., 0., ..., 1., 0., 0.],
        [0., 1., 0., ..., 1., 1., 0.]],

       [[1., 1., 0., ..., 1., 0., 0.],
        [0., 1., 0., ..., 1., 1., 0.],
        [1., 0., 0., ..., 1., 1., 0.]],

       ...,

       [[0., 0., 1., ..., 1., 0., 0.],
        [1., 1., 1., ..., 1., 1., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 1., 1., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 0.],
        [0., 1., 0., ..., 1., 0., 0.]]], dtype=float32)